In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup as soup
from lxml import etree
import csv
import unicodedata

In [3]:
colnames=['IDN'] 

#Einlesen einer Datei mit Identifikationsnummern (IDNs) und Umwandlung der Spalte IDN in Liste: 
data = pd.read_csv("IDN_SRU.csv", names=colnames, header=None)
idns = data['IDN'].to_list()
print(idns)

['1015015255', '1028850050', '1032490276', '1032533099', '1021093599', '103533433X', '1112959122', '101729822X', '1132586577', '1032739355', '1032767553', '1124866795', '1068878762', '1071831992', '1070175900', '1015018475', '1035742039', '103143237X', '1131664930', '1031677909', '1035342960', '1196537798', '1032489677', '1031434348', '1014155843', '1031691375', '1031435522', '1031434879', '1048030954', '1031436111']


In [4]:
#gekürzte Funktion zur Abfrage der IDNs aus dem SRU-Tutorial (www.dnb.de/dnblabtutorials)
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, 'lxml')
    records = xml.find_all('record', {'type':'Bibliographic'})
          
    return records
    
    

In [5]:
#Übergabe der einzelnen Elemente aus der IDN-Liste an die Funktion, um für jedes Element eine SRU-Abfrage 
#auszulösen. Die Ergebnisse der einzelnen Abfragen werden als Liste in der Variable "response" gespeichert
response = [dnb_sru(searchtext) for searchtext in idns]

#Ausgabe der Anzahl der in der Liste enthaltenen Elemente - diese sollte mit der Anzahl der abgefragten
#IDNs übereinstimmen. 
print(len(response)) 

30


In [6]:
#Funktion, um aus den einzelnen Records bestimmte Inhalte zu extrahieren: 

def parse_record(record):
    
    record = record[0]
        
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
    meta_dict = {"idn":idn,
                 "titel":titel}
    
    return meta_dict

In [167]:
#Übergabe der einzelnen Records an die Funktion und Ausgabe als DataFrame:
output = [parse_record(record) for record in response]
df = pd.DataFrame(output)
df

,idn,titel
0,1015015255,Das tragische Schicksal der Zarenfamilie
1,1028850050,"Statistik, Presse und Organisationen des Juden..."
2,1032490276,Die Geschichte eines Hochverräters
3,1032533099,Die Lösung sozialer Konflikte
4,1021093599,Die drei Reiche
5,103533433X,Jedermann Selbstversorger!
6,1112959122,Über die Aufgabe des Geschichtsschreibers
7,101729822X,Landwirtschaftliches Güter-Adressbuch für die ...
8,1132586577,Bayreuthisches Pfarrerbuch
9,1032739355,Der Gott der Makkabäer
